In [340]:
import pandas as pd
from math import radians, sin, cos, sqrt, atan2

#### Funções para o cálculo de distância entre uma estação e outra no DataFrame

In [341]:
def haversine_distance(lat1, lon1, lat2, lon2):
    R = 6371.0 * 1000

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlon / 2) ** 2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return round(distance, 2)


def get_station_info(station_name, df):
    station_rows = df[df["Station"] == station_name]
    if not station_rows.empty:
        station_info = {
            "Latitude": station_rows.iloc[0]["Latitude"],
            "Longitude": station_rows.iloc[0]["Longitude"],
            "Lines": set(station_rows["Line"]),
        }
        return station_info
    else:
        return None


def calculate_distance(row, df):
    origin_info = get_station_info(row["Station"], df)
    dest_info = get_station_info(row["Destination"], df)

    if origin_info and dest_info:
        if origin_info["Lines"].intersection(dest_info["Lines"]):
            return haversine_distance(
                origin_info["Latitude"],
                origin_info["Longitude"],
                dest_info["Latitude"],
                dest_info["Longitude"],
            )
    return None

#### Formatações do Dataframe

In [349]:
df = pd.read_csv("Metro_Madrid_2021.csv")
df = df.drop("Traffic", axis=1)
df["Longitude"] = df["Longitude"].str.replace(",", ".").astype(float)
df["Latitude"] = df["Latitude"].str.replace(",", ".").astype(float)
df.insert(1, "Destination", df["Station"].shift(-1))
df.insert(2, "Distance", df.apply(lambda row: calculate_distance(row, df), axis=1))
df = df.dropna(subset=["Distance"])
df.drop(df.loc[df["Distance"] > 5000].index, inplace=True)
new_rows = df.copy()
new_rows.rename(
    columns={"Station": "Destination", "Destination": "Station"}, inplace=True
)
df = pd.concat([df, new_rows], ignore_index=True)

,Station,Destination,Distance,Line,Order of Points,Longitude,Latitude


#### Criação do grafo

In [350]:
graph_df = df[["Station", "Destination", "Distance"]]
graph_df = graph_df.dropna()

In [351]:
graph = {}
#Colocando todos dos dados do DataFrame em um Dict
for row in df.itertuples():
    station = row.Station
    destination = row.Destination
    distance = row.Distance

    if pd.isna(distance):
        continue

    if station not in graph:
        graph[station] = []

    graph[station].append((destination, distance))

## Depth First Search (Busca Cega)

In [353]:
def dfs(graph, station, destination, visited, path):
    if visited == []:
        visited = []
    if path == []:
        path = [station]

    visited.append(station)

    #Se o nó de destino já foi alcançado, o caminho é apenas ele
    if station == destination:
        return path

    #Verificando os nós vizinhos
    adjacents = [adjacent for adjacent, distance in graph.get(station, [])]
    for adjacent in adjacents:
        if adjacent not in visited:
            new_path = dfs(graph, adjacent, destination, visited, path + [adjacent])
            if new_path:  #Se um caminho válido foi encontrado, retorne-o
                return new_path

    return None  #Se nenhum caminho foi encontrado

In [354]:
start = "Bambú"
end = "Santiago Bernabéu"
dfs_result_dict = dfs(graph, start, end, [], [])
dfs_result_dict

['Bambú',
 'Chamartín',
 'Plaza de Castilla',
 'Valdeacederas',
 'Tetuán',
 'Estrecho',
 'Alvarado',
 'Cuatro Caminos',
 'Ríos Rosas',
 'Iglesia',
 'Bilbao',
 'Tribunal',
 'Gran Vía',
 'Sol',
 'Tirso de Molina',
 'Antón Martín',
 'Estación del Arte',
 'Atocha',
 'Menéndez Pelayo',
 'Pacífico',
 'Méndez Álvaro',
 'Arganzuela-Planetario',
 'Legazpi',
 'Usera',
 'Plaza Elíptica',
 'Opañel',
 'Oporto',
 'Urgel',
 'Marqués de Vadillo',
 'Pirámides',
 'Acacias',
 'Puerta de Toledo',
 'La Latina',
 'Ópera',
 'Santo Domingo',
 'Noviciado',
 'San Bernardo',
 'Quevedo',
 'Canal',
 'Alonso Cano',
 'Gregorio Marañón',
 'Avenida de América',
 'Prosperidad',
 'Alfonso XIII',
 'Avenida de la Paz',
 'Arturo Soria',
 'Esperanza',
 'Canillas',
 'Mar de Cristal',
 'Pinar del Rey',
 'Colombia',
 'Nuevos Ministerios',
 'Santiago Bernabéu']